In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import pymc3 as pm


# Define your observed data (N and P time series)
def model(y,t,p2):
    derivs=y**0.5*p2-1
    return derivs

t=np.linspace(0,500,501)
y0=10
sol=odeint(model,y0,t,args=(0.5,))
plt.plot(t,sol);plt.show()
observed_N=sol


# Define the priors for model parameters
with pm.Model() as model:
    p = pm.Normal("r", mu=1, sd=1)  # Prior for intrinsic growth rate
    
    # Define the initial conditions for N and P
    Y0 = pm.Normal("N0", mu=10, sd=1)
    
    # Define the differential equations
    def lotka_volterra(Y, t,p):
        return Y**0.5*p-1
    
    # Use the ODE solver to compute the predicted N and P values
    sol_pred= odeint(model,Y0,t,args=(p,))
    
    # Likelihood function (observation model)
    N_obs = pm.Normal("N_obs", mu=sol_pred[:,0], sd=1, observed=observed_N)


with model:
    trace = pm.sample(1000, tune=1000, cores=4)
    
    
pm.traceplot(trace)
plt.show()

pm.summary(trace)

# Plot posterior predictive checks
ppc = pm.sample_posterior_predictive(trace, samples=500, model=model)
plt.plot(observed_N, label="Observed N")
plt.plot(observed_P, label="Observed P")
plt.plot(np.mean(ppc['N_obs'], axis=0), label="Predicted N (Mean)")
plt.plot(np.mean(ppc['P_obs'], axis=0), label="Predicted P (Mean)")
plt.legend()
plt.show()
    